In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-summarization/Amazon Product Reviews.csv


In [5]:
import re

In [6]:
df = pd.read_csv("/kaggle/input/text-summarization/Amazon Product Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
df = df.dropna()

In [8]:
import torch
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [9]:
df = df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator',
                       'Score', 'Time'], 1)
df = df.reset_index(drop = True)

In [10]:
df.head(10)

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...
5,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,Yay Barley,Right now I'm mostly just sprouting this so my...
9,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [11]:
for i in range(5):
    print("Review ", i+1, ':')
    print('Summary: ', df['Summary'][i])
    print('Text: ', df['Text'][i])
    print('\n')

Review  1 :
Summary:  Good Quality Dog Food
Text:  I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


Review  2 :
Summary:  Not as Advertised
Text:  Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".


Review  3 :
Summary:  "Delight" says it all
Text:  This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Wi

In [12]:
contractions = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not",
    "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
    "he'd've": "he would have", "he'll": "he will", "he's": "he is", "how'd": "how did", "how'll": "how will", "how's": "how is",
    "i'd": "i would", "i'll": "i will", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
    "it'll": "it will", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
    "mightn't": "might not", "must've": "must have", "mustn't": "must not", "needn't": "need not", "oughtn't": "ought not",
    "shan't": "shall not", "sha'n't": "shall not", "she'd": "she would", "she'll": "she will","she's": "she is", "should've": "should have", 
    "shouldn't": "should not", "that'd": "that would", "that's": "that is", "there'd": "there had", "there's": "there is",
    "they'd": "they would", "they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not",
    "we'd": "we would", "we'll": "we will", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
    "what're": "what are", "what's": "what is", "what've": "what have", "where'd": "where did", "where's": "where is",
    "who'll": "who will", "who's": "who is", "won't": "will not", "wouldn't": "would not", "you'd": "you would",
    "you'll": "you will", "you're": "you are"
}

In [13]:
from nltk.corpus import stopwords

In [14]:
def clean_text(text, remove_stopwords = True):
    text = text.lower()
    
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = ' '.join(new_text)
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]
        text = " ".join(text)
    
    return text

We will remove the stopwords from the reviews but keep them in the summaries so that they sound like natural phrases

In [15]:
# Clean the summaries and texts
clean_summaries = []
for summary in df['Summary']:
    clean_summaries.append(clean_text(summary, remove_stopwords = False))
print("Summaries are complete.")

clean_texts = []
for text in df['Text']:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [16]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
good quality dog food
bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better

Clean Review # 2
not as advertised
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo

Clean Review # 3
 delight  says it all
confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch

Clean Review # 4
cough medicine
looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal

Clean Review # 5
great taffy
great taffy great price wide assortment yummy taffy delivery quick taffy lover deal



In [17]:
count_dict = {}
def count_words(text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [18]:
count_words(clean_summaries)
count_words(clean_texts)

In [19]:
len(count_dict)

132884

In [20]:
!pip install --upgrade tensorflow==1.15

     |████████████████████████████████| 412.3 MB 22 kB/s s eta 0:00:01   |█                               | 12.8 MB 6.8 MB/s eta 0:00:59     |█████████████████████████████   | 374.7 MB 42.2 MB/s eta 0:00:01     |█████████████████████████████▎  | 376.5 MB 42.2 MB/s eta 0:00:01     |██████████████████████████████  | 385.2 MB 42.2 MB/s eta 0:00:01     |██████████████████████████████▌ | 392.3 MB 18.3 MB/s eta 0:00:02
     |████████████████████████████████| 503 kB 21.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 42.5 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=94295d99578262a2809f7f1c4bfdcbe8b47398dbeb357bf188c444459f689094
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.

In [21]:
import tensorflow as tf
tf.__version__

'1.15.0'

I had to downgrade the version of tensorflow because elmo is not compatible with tf2.x

In [24]:
print(clean_texts[50], clean_summaries[50], sep = '\n')

oatmeal good mushy soft like quaker oats way go
do not like it


Torch Req

In [26]:
from __future__ import unicode_literals, print_function, division
from io import open
import random
import string
import unidecode

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F

In [27]:
glove_path = '../input/glove-200d/glove.6B.200d.txt'

In [28]:
#creating a dictionary of sorts for the entire corpus
class Language:
    def __init__(self, name):
        self.name = name
        self.word2idx = {}
        self.wordcnt = {}
        self.n_words = 2 #counting sos and eos initially
        self.idx2word = {0 : 'SOS', 1 : 'EOS'} #Start of sentence and end of sentence respectively.
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.n_words
            self.idx2word[self.n_words] = word
            self.n_words += 1
            self.wordcnt[word] = 1
        else:
            self.wordcnt[word] += 1

In [29]:
def readText(text, summary):
    print("Reading text...")
    pairs = [[text[i], summary[i]] for i in range(len(text))]
    
    input_text = Lang(text)
    output_text = Lang(summary)
    return input_text, output_text, pairs

In [30]:
def prepareData(text, summary):
    input_text, output_text, pairs = readText(text, summary)
    
    print("Read %s sentence pairs", len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_text.addSentence(pairs[0])
        output_text.addSentence(pairs[1])
    print("Finished counting words:")
    print(input_text.name, input_text.n_words)
    print(output_text.name, output_text.n_words)
    return input_text, output_text, pairs